In [2]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
from numpy import array


     |████████████████████████████████| 6.4 MB 11.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 3.0 MB/s 


In [3]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
  'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in','all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(12,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-12][0],
                df[each-12][1],
                df[each-12][2],
                df[each-12][3],
                df[each-12][4],
                df[each-11][0],
                df[each-11][1],
                df[each-11][2],
                df[each-11][3],
                df[each-11][4],
                df[each-10][0],
                df[each-10][1],
                df[each-10][2],
                df[each-10][3],
                df[each-10][4],
                df[each-9][0],
                df[each-9][1],
                df[each-9][2],
                df[each-9][3],
                df[each-9][4],
                df[each-8][0],
                df[each-8][1],
                df[each-8][2],
                df[each-8][3],
                df[each-8][4],
                df[each-7][0],
                df[each-7][1],
                df[each-7][2],
                df[each-7][3],
                df[each-7][4],
                df[each-6][0],
                df[each-6][1],
                df[each-6][2],
                df[each-6][3],
                df[each-6][4],
                df[each-5][0],
                df[each-5][1],
                df[each-5][2],
                df[each-5][3],
                df[each-5][4],
                df[each-4][0],
                df[each-4][1],
                df[each-4][2],
                df[each-4][3],
                df[each-4][4],
                df[each-3][0],
                df[each-3][1],
                df[each-3][2],
                df[each-3][3],
                df[each-3][4],
                df[each-2][0],
                df[each-2][1],
                df[each-2][2],
                df[each-2][3],
                df[each-2][4],
                df[each-1][0],
                df[each-1][2],
                df[each-1][1],
                df[each-1][3],
                df[each-1][4],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(1,10))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled)
  df_scaled["suggestion"] = last_column
  return df_scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      #df = column_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df   

In [31]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

symbols = get_crypto_syms()
#symbols = read_syms_from_txt()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 13:
          dfs.append(df)

data = process(dfs)
xTrain, xTest, yTrain, yTest = spliting(data)
data


1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-GBP: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-GBP: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-GBP: No data found for this date range, symbol may be delisted
Passing.

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,suggestion
0,9.706879,10.000000,8.875120,9.455215,9.455215,9.401446,9.389876,4.302419,5.595268,5.595268,...,4.718768,3.646289,3.895306,3.895306,3.889629,1.000000,4.068858,1.455729,1.455729,0.0
1,10.000000,9.987671,4.566510,5.944161,5.944161,5.903536,6.497416,1.128067,2.653120,2.653120,...,4.317629,1.047474,1.533096,1.533096,1.501734,1.000000,2.533964,1.438175,1.438175,1.0
2,6.625815,7.307172,1.146930,2.896621,2.896621,2.879508,6.861783,2.213081,4.859447,4.859447,...,2.759913,1.000000,1.502717,1.502717,1.591258,1.140889,3.440285,2.981205,2.981205,0.0
3,2.879508,6.861783,2.213081,4.859447,4.859447,4.745500,5.349456,2.671929,3.400729,3.400729,...,3.440285,1.140889,2.981205,2.981205,3.049874,2.129668,4.074134,2.470487,2.470487,0.0
4,4.950069,5.530511,2.957227,3.657653,3.657653,3.694922,4.928180,3.491203,4.465726,4.465726,...,4.304841,2.436078,2.763627,2.763627,2.814560,1.000000,2.685568,1.305905,1.305905,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769662,6.421697,8.116731,5.988977,7.235217,7.235217,7.239544,7.407686,6.038740,7.351123,7.351123,...,3.735730,3.243973,3.243973,3.243973,3.248918,1.000000,3.490625,1.077273,1.077273,1.0
1769663,7.239544,7.407686,6.038740,7.351123,7.351123,7.340306,9.326189,7.229653,8.340820,8.340820,...,3.490625,1.000000,1.077273,1.077273,1.074183,1.074183,3.296518,2.758706,2.758706,1.0
1769664,7.340306,9.326189,7.229653,8.340820,8.340820,8.336803,10.000000,7.392851,7.905318,7.905318,...,3.296518,1.074183,2.758706,2.758706,2.759015,2.730270,3.597877,3.333917,3.333917,0.0
1769665,8.336803,10.000000,7.392851,7.905318,7.905318,7.900062,7.937153,5.943541,7.084381,7.084381,...,3.597877,2.730270,3.333917,3.333917,3.330209,2.618691,3.620132,2.947867,2.947867,1.0


In [8]:
model = Sequential()


model.add(Dense(5000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2500, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5000)              305000    
                                                                 
 dense_1 (Dense)             (None, 2500)              12502500  
                                                                 
 dense_2 (Dense)             (None, 2000)              5002000   
                                                                 
 dense_3 (Dense)             (None, 1000)              2001000   
                                                                 
 dense_4 (Dense)             (None, 500)               500500    
                                                                 
 dense_5 (Dense)             (None, 250)               125250    
                                                                 
 dense_6 (Dense)             (None, 100)              

In [9]:
model.fit(xTrain,yTrain,epochs=50,batch_size=50000,validation_data=(xTest,yTest))

Epoch 1/50
 2/32 [>.............................] - ETA: 42:37 - loss: 2.1963 - accuracy: 0.4993

KeyboardInterrupt: ignored

In [1]:
model.save("9474_BinaryTest_v08.h5")

NameError: ignored

In [7]:
from google.colab import drive
drive.mount("/content/drive")

data = pd.read_csv("/content/drive/MyDrive/Colab Files/BinaryTest_v08.csv")
data = data.drop("Unnamed: 0",axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1592700, 60) (1592700,)
(176967, 60) (176967,)


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,suggestion
0,9.706879,10.000000,8.875120,9.455215,9.455215,9.401446,9.389876,4.302419,5.595268,5.595268,...,4.718768,3.646289,3.895306,3.895306,3.889629,1.000000,4.068858,1.455729,1.455729,0.0
1,10.000000,9.987671,4.566510,5.944161,5.944161,5.903536,6.497416,1.128067,2.653120,2.653120,...,4.317629,1.047474,1.533096,1.533096,1.501734,1.000000,2.533964,1.438175,1.438175,1.0
2,6.625815,7.307172,1.146930,2.896621,2.896621,2.879508,6.861783,2.213081,4.859447,4.859447,...,2.759913,1.000000,1.502717,1.502717,1.591258,1.140889,3.440285,2.981205,2.981205,0.0
3,2.879508,6.861783,2.213081,4.859447,4.859447,4.745500,5.349456,2.671929,3.400729,3.400729,...,3.440285,1.140889,2.981205,2.981205,3.049874,2.129668,4.074134,2.470487,2.470487,0.0
4,4.950069,5.530511,2.957227,3.657653,3.657653,3.694922,4.928180,3.491203,4.465726,4.465726,...,4.304841,2.436078,2.763627,2.763627,2.814560,1.000000,2.685568,1.305905,1.305905,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769662,6.421697,8.116731,5.988977,7.235217,7.235217,7.239544,7.407686,6.038740,7.351123,7.351123,...,3.735730,3.243973,3.243973,3.243973,3.248918,1.000000,3.490625,1.077273,1.077273,1.0
1769663,7.239544,7.407686,6.038740,7.351123,7.351123,7.340306,9.326189,7.229653,8.340820,8.340820,...,3.490625,1.000000,1.077273,1.077273,1.074183,1.074183,3.296518,2.758706,2.758706,1.0
1769664,7.340306,9.326189,7.229653,8.340820,8.340820,8.336803,10.000000,7.392851,7.905318,7.905318,...,3.296518,1.074183,2.758706,2.758706,2.759015,2.730270,3.597877,3.333917,3.333917,0.0
1769665,8.336803,10.000000,7.392851,7.905318,7.905318,7.900062,7.937153,5.943541,7.084381,7.084381,...,3.597877,2.730270,3.333917,3.333917,3.330209,2.618691,3.620132,2.947867,2.947867,1.0


In [34]:
from google.colab import drive
drive.mount("/content/drive")

data.to_csv("BinaryTest_v08.csv")

Mounted at /content/drive
